In [19]:
import torch
import logging
from trl import SFTTrainer
from datasets import load_from_disk
from transformers import TrainingArguments
from unsloth import FastLanguageModel, is_bfloat16_supported
from dotenv import load_dotenv
from utils import log_metrics_and_confusion_matrices_wandb, plot_count_and_normalized_confusion_matrix, process_output
import os
import wandb
import pandas as pd
import re

In [17]:
load_dotenv()
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
WANDB_PROJECT_NAME = 'llm-classification'
experiment_name = 'llama_fine_tuning'
model_name = huggingface_model_name = "dariast/Meta-Llama-3.1-8B-Instruct"
dataset_name = 'euvsdisinfo'
api_key = os.getenv('API_KEY')
base_url = os.getenv('BASE_URL')

In [ ]:
from huggingface_hub import notebook_login
HF_TOKEN = os.getenv("HF_TOKEN")
notebook_login() 

In [ ]:
wandb.login() 
wandb.init(
    # set the wandb project where this run will be logged
    project=WANDB_PROJECT_NAME,
    name=experiment_name,
    # track hyperparameters and run metadata
    config = {
        "model": model_name,
        "dataset": dataset_name
    }
    )

## Load data

In [13]:
# Load the dataset
dataset = load_from_disk("/home/dariast/Disinfo_Russian_Media/data/fine_tuning_dataset")
# Access train and test splits
train_ds = dataset['train']
test_ds = dataset['test']

In [3]:
max_seq_length = 2048 
dtype = None 
load_in_4bit = False 

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",    

    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None 
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


lr1 = 2e-4


In [15]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 4, # Set this for 1 full training run.
        # max_steps = 40,
        learning_rate = 3e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [16]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA H100 PCIe. Max memory = 79.109 GB.
5.906 GB of memory reserved.


## Train

In [ ]:
trainer_stats = trainer.train()

In [18]:
# Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

840.0644 seconds used for training.
14.0 minutes used for training.
Peak reserved memory = 9.158 GB.
Peak reserved memory for training = 3.252 GB.
Peak reserved memory % of max memory = 11.576 %.
Peak reserved memory for training % of max memory = 4.111 %.


## Save

In [ ]:
# HF save
model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged(huggingface_model_name, tokenizer, save_method = "merged_16bit", token = HF_TOKEN)

In [21]:
# Local save
model.save_pretrained("llama_fine_tuned") # Local saving
tokenizer.save_pretrained("llama_fine_tuned")

README.md:   0%|          | 0.00/606 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

Saved model to https://huggingface.co/dariast/Meta-Llama-3.1-8B-4bit-python


In [ ]:
model.push_to_hub_gguf(
    "dariast/Meta-Llama-3.1-8B-Instruct", # Change hf to your username!
    tokenizer,
    quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
    token = HF_TOKEN,
)

## Testing

In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "dataset/llama_fine_tuned", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype, 
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [14]:
test_ds['text']
def substitute_last_part_of_prompt(input_string):
    result = re.sub(r'\n{3}"Class": .*?<\|eot_id\|>', '', input_string)
    return result
test_prompts = [substitute_last_part_of_prompt(prompt) for prompt in test_ds['text']]
log_dataframe = pd.DataFrame(test_ds)
model_name = 'llama_fine_tuned'

In [ ]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

In [ ]:
predictions = []
# Iterate through all test instances
for i, prompt in enumerate(test_prompts):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    generated_output = model.generate(**inputs, streamer=text_streamer, max_new_tokens=10)
    predicted_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)
    predictions.append(predicted_text)

In [ ]:
processed_outputs = []
for output in predictions:
    print(f"Initial output: {output}")  
    processed_outputs.append(process_output(output))

In [ ]:
# Add the outputs to the dataframe
log_dataframe[f"predicted_{model_name}"] = processed_outputs

# Log the dataset output to wandb
predictions_artifact = wandb.Artifact('predictions', type='outputs')
with predictions_artifact.new_file(f'predictions_{model_name}.csv', mode='w', encoding='utf-8') as f:
    log_dataframe.to_csv(f)
wandb.run.log_artifact(predictions_artifact)

In [24]:
# Clean true labels 
labels = log_dataframe['OUTPUT']
processed_true_labels = [process_output(label) for label in labels]

In [ ]:
task_name = 'llama_fine_tuned'
# Measure performance 
try:    
    y_true = processed_true_labels
    y_pred = processed_outputs

    cm_plot_path, classification_report, metrics = plot_count_and_normalized_confusion_matrix(y_true=y_true, y_pred=y_pred, save_path="./img", figure_title=f'{task_name}_{model_name}')
    
    log_metrics_and_confusion_matrices_wandb(cm_plot_path=cm_plot_path, classification_report=classification_report, metrics=metrics, task_name=f'{task_name}')

except Exception as e:
    print('Error computing metrics: ', e)

# Finish logging
wandb.finish()